#Запуск алгоритма

In [3]:
#@title Установка пакетов
!pip  install langchain==0.0.335 openai==1.2.3 tiktoken==0.5.1 pydantic==1.10.8 faiss-cpu==1.7.4 nltk oauth2client >/dev/null



In [4]:
#@title Импорт библиотек

import os
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import openai
from openai import OpenAI
import tiktoken
import re
import requests
from langchain.docstore.document import Document


In [13]:
#@title Определим функции

def load_googledoc_by_url(doc_url) -> str: # Функция load_googledoc_by_url предназначена для загрузки текста из гуглдока, по ссылке (doc_url)
  # Extract the document ID from the URL
  match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', doc_url)
  if match_ is None:
    raise ValueError('Invalid Google Docs URL')
  doc_id = match_.group(1)

  # Download the document as plain text
  response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
  response.raise_for_status()
  return response.text

def load_document_text(file_path) -> str:   # Функция load_document_text предназначена для загрузки текста из файла, расположенного по указанному пути (file_path)
#    with open(file_path, 'r', encoding='windows-1251') as file:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    text_encoded = text.encode('utf-8')
    text = text_encoded.decode('utf-8')
    return text


def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

def split_text(text, max_count, chunk_overlap, verbose=0):
    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
                          ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    fragments = markdown_splitter.split_text(text)

    # Создание объекта разделителя текста
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:

        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = num_tokens(fragment.page_content)
            print(f"Tokens in text fragment = {count}\n{'-' * 5}\n{fragment.page_content}\n{'=' * 20}")

        # Разбиение фрагмента текста на части заданной длины с помощью разделителя
        # и добавление каждой части в список source_chunks  и передача в чанк метадата из маркдауновскго сплиттера
        source_chunks.extend(Document(page_content=chunk, metadata=fragment.metadata) for chunk in splitter.split_text(fragment.page_content))

    # Возвращение списка фрагментов текста
    return source_chunks

def create_search_index(data, chunk_size, chunk_overlap, verbo):
    source_chunks = []
    source_chunks = split_text(text=data, max_count=chunk_size, chunk_overlap=chunk_overlap, verbose=verbo)
    return FAISS.from_documents(source_chunks, OpenAIEmbeddings())

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start> assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")


def answer_user_question(system_doc_text, knowledge_base_url, topic, instructions, temperature, verbose, k, chunk_size, chunk_overlap, model):
    knowledge_base_text = load_googledoc_by_url(knowledge_base_url)
    knowledge_base_index = create_search_index(knowledge_base_text, chunk_size, chunk_overlap, verbose)
    return answer_index(system_doc_text, topic, instructions, knowledge_base_index, temperature, verbose, k, model)


def answer_index(system, topic, instructions, search_index, temp, verbose, k, model):
    docs = search_index.similarity_search_with_score(topic, k=k)
    message_content = '\n '.join([f'Отрывок текста №{i+1}\n{doc[0].page_content}' for i, doc in enumerate(docs)])
    messages = [{"role": "system", "content": system}, {"role": "user", "content": f"{instructions}\n\nТексты для анализа:\n{message_content}"}]

    completion = openai.chat.completions.create(model=model, messages=messages, temperature=temp)
    return completion.choices[0].message.content



In [7]:
#@title Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Тест моделей

In [17]:
#@title 1. Ресепшн
model = "gpt-3.5-turbo-1106" #@param ["gpt-3.5-turbo", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-1106"]
temperature = 0.8 #@param {type: "slider", min: 0, max: 1, step:0.1}
system_path = 'System_1.txt' #@param {type:"string"}
#system_prompt = "\u0412\u044B \u043E\u043F\u044B\u0442\u043D\u044B\u0439 \u0438 \u0432\u043D\u0438\u043C\u0430\u0442\u0435\u043B\u044C\u043D\u044B\u0439 \u0430\u0434\u043C\u0438\u043D\u0438\u0441\u0442\u0440\u0430\u0442\u043E\u0440 \u0441\u0430\u043B\u043E\u043D\u0430 \u043A\u0440\u0430\u0441\u043E\u0442\u044B. \u041A \u0412\u0430\u043C \u0432 \u043C\u0435\u0441\u0441\u0435\u043D\u0434\u0436\u0435\u0440\u0435 \u0432 \u0441\u043E\u0446\u0441\u0435\u0442\u0438 \u043E\u0431\u0440\u0430\u0449\u0430\u0435\u0442\u0441\u044F \u043A\u043B\u0438\u0435\u043D\u0442." #@param {type:"string"}
instructions_path = "instruction_1.txt" #@param {type:"string"}
#instructions = "\u041F\u043E\u0436\u0430\u043B\u0443\u0439\u0441\u0442\u0430, \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0438\u0442\u0435, \u044D\u0442\u043E \u043D\u043E\u0432\u044B\u0439 \u043A\u043B\u0438\u0435\u043D\u0442, \u043A\u043E\u0442\u043E\u0440\u043E\u043C\u0443 \u043D\u0443\u0436\u043D\u043E \u0434\u0430\u0442\u044C \u043E\u043F\u0438\u0441\u0430\u043D\u0438\u0435 \u0443\u0441\u043B\u0443\u0433 \u0412\u0430\u0448\u0435\u0433\u043E \u0441\u0430\u043B\u043E\u043D\u0430, \u0438\u043B\u0438 \u044D\u0442\u043E \u0412\u0430\u0448 \u043F\u043E\u0441\u0442\u043E\u044F\u043D\u043D\u044B\u0439 \u043A\u043B\u0438\u0435\u043D\u0442, \u043A\u043E\u0442\u043E\u0440\u043E\u043C\u0443 \u043D\u0443\u0436\u043D\u043E \u0437\u0430\u043F\u0438\u0441\u0430\u0442\u044C\u0441\u044F \u043D\u0430 \u043F\u0440\u043E\u0446\u0435\u0434\u0443\u0440\u0443. \u041E\u0442\u0432\u0435\u0442 \u0434\u0430\u0439 \u0432 \u0444\u043E\u0440\u043C\u0430\u0442\u0435: % \u0443\u0432\u0435\u0440\u0435\u043D\u043D\u043E\u0441\u0442\u0438 \u0447\u0442\u043E \u043A\u043B\u0438\u0435\u043D\u0442 \u043D\u043E\u0432\u044B\u0439/ % \u0443\u0432\u0435\u0440\u0435\u043D\u043D\u043E\u0441\u0442\u0438 \u0447\u0442\u043E \u043A\u043B\u0438\u0435\u043D\u0442 \u0441\u0442\u0430\u0440\u044B\u0439" #@param {type:"string"}
topicphrase = '\u0414\u043E\u0431\u0440\u044B\u0439 \u0434\u0435\u043D\u044C, \u0443\u0432\u0438\u0434\u0435\u043B \u043E\u0431\u044A\u044F\u0432\u043B\u0435\u043D\u0438\u0435 \u043E \u0432\u0430\u0448\u0435\u043C \u0441\u0430\u043B\u043E\u043D\u0435, \u0445\u043E\u0447\u0443 \u043F\u043E\u0437\u043D\u0430\u043A\u043E\u043C\u0438\u0442\u044C\u0441\u044F \u0441 \u0432\u0430\u0448\u0438\u043C\u0438 \u0443\u0441\u043B\u0443\u0433\u0430\u043C\u0438' #@param {type:"string"}
system_prompt = load_document_text(system_path)
instructions = load_document_text(instructions_path)
#print(instructions, " \n", system_prompt)
messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f"{instructions}\n\nСообщение клиента:\n{topicphrase}"}]
completion = openai.chat.completions.create(model=model, messages=messages, temperature=temperature)
output1 = completion.choices[0].message.content
print("\nОтвет:\n", output1)

Пожалуйста, определите, это новый клиент, которому нужно дать описание услуг Вашего салона, или это Ваш постоянный клиент, которому нужно записаться на процедуру. Ответ дай в формате: % уверенности что клиент новый/ % уверенности что клиент старый  
 Вы опытный и внимательный администратор салона красоты. К Вам в мессенджере в соцсети обращается клиент.

Ответ:
 По данному сообщению я могу предположить, что это новый клиент. Я уверен на 80% что клиент новый / 20% что клиент старый


In [ ]:
#@title 2. Тест по вопросам к БЗ (новый клиент)
model = "gpt-3.5-turbo-1106" #@param ["gpt-3.5-turbo", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-1106"]
kb_url = "https://docs.google.com/document/d/1GpFWeAMnrxW5araflGmu0kZozkd0w9-afv1wyKmhO0M" #@param {type:"string"}
chunk_size = 608 #@param {type: "slider", min: 200, max: 1024, step:8}
chunk_overlap = 0 #@param {type: "slider", min: 0, max: 256, step:8}
temperature = 0.4 #@param {type: "slider", min: 0, max: 1, step:0.1}
num_fragment = 6 #@param {type:"integer"}
verbose = "1" #@param [0,1]
system_prompt = "\u0422\u044B \u0441\u0443\u043F\u0435\u0440 \u0441\u043F\u0435\u0446\u0438\u0430\u043B\u0438\u0441\u0442 \u043F\u043E \u043B\u0430\u0437\u0435\u0440\u043D\u043E\u0439 \u044D\u043F\u0438\u043B\u044F\u0446\u0438\u0438 \u0438 \u043A\u043E\u043D\u0441\u0443\u043B\u044C\u0442\u0430\u043D\u0442 \u043F\u043E \u043A\u043E\u0441\u043C\u0435\u0442\u0438\u043A\u0435. \u0422\u0435\u0431\u044F \u0441\u043F\u0440\u0430\u0448\u0438\u0432\u0430\u0435\u0442 \u043A\u043B\u0438\u0435\u043D\u0442 \u043E \u043F\u0440\u043E\u0434\u0443\u043A\u0446\u0438\u0438 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438." #@param {type:"string"}
instructions = "\u041E\u0442\u0432\u0435\u0442\u044C \u043D\u0430 \u0432\u043E\u043F\u0440\u043E\u0441 \u043A\u043B\u0438\u0435\u043D\u0442\u0430 \u043A\u0430\u043A \u043C\u043E\u0436\u043D\u043E \u043F\u043E\u0434\u0440\u043E\u0431\u043D\u0435\u0435, \u043D\u0438\u0447\u0435\u0433\u043E \u043D\u0435 \u043F\u0440\u0438\u0434\u0443\u043C\u044B\u0432\u0430\u0439 \u043E\u0442 \u0441\u0435\u0431\u044F" #@param {type:"string"}
Вопрос_клиента = "\u041A\u0430\u043A\u043E\u0439 \u0441\u043E\u0441\u0442\u0430\u0432 \u0443 \u043C\u043E\u043B\u043E\u0447\u043A\u0430 \u0441 \u044D\u0444\u0444\u0435\u043A\u0442\u043E\u043C \u043B\u0438\u0444\u0442\u0438\u043D\u0433\u0430?" #@param {type:"string"}
#content_base = load_googledoc_by_url(kb_url)

output1 = answer_user_question(system_prompt, kb_url, Вопрос_клиента,
                               instructions, temperature, int(verbose), num_fragment,
                               chunk_size, chunk_overlap, model) #ОБЩИЙ

print("\nОтвет:\n", output1)

Tokens in text fragment = 605
-----
﻿#Косметика собственного производства laser love  
Косметика собственного производства laser love.  
Косметика Laser Love производится в Новосибирском Академгородке. Совместно с профессионалами из Научно-исследовательского центра Сибири. Наши партнеры занимаются разработкой рецептуры по пожеланиям заказчика, имеют научно-техническую базу и современное производство. Высокотехнологичное оборудование исключает нагревание компонентов и сохраняет полезные свойства.
Косметика Laser Love производится из натуральных компонентов.
Laser Love отказались от бесполезных основ, искусственных консервантов и комедогенных масел.
Перечень продукции собственного производства laser love:
* Молочко после эпиляции Грейпфрут.
* Гель для укладки бровей.
* Крем питательный для рук Пассия и Росянка.
* Крем увлажняющий для тела Грейпфрут.
* Молочко для тела с лифтинг эффектом с доз.Нина.
* Молочко для тела мультивитамин с доз. Смородина.
* Кокосовое масло.
* Масло массажн